# LIBRERÍAS

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import os
import xgboost 
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from scipy.stats import randint, uniform

In [2]:
conexion=sqlite3.connect("C:\Proyecto-API-Hoteles/Data/hotel_bookings_clean.db")
df=pd.read_sql_query("select * from hotel_bookings_clean",conexion)
conexion.close()

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84080 entries, 0 to 84079
Data columns (total 28 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   hotel                           84080 non-null  object 
 1   lead_time                       84080 non-null  int64  
 2   arrival_date_month              84080 non-null  object 
 3   arrival_date_week_number        84080 non-null  int64  
 4   arrival_date_day_of_month       84080 non-null  int64  
 5   stays_in_weekend_nights         84080 non-null  int64  
 6   stays_in_week_nights            84080 non-null  int64  
 7   adults                          84080 non-null  int64  
 8   children                        84080 non-null  float64
 9   babies                          84080 non-null  int64  
 10  meal                            84080 non-null  object 
 11  country                         84080 non-null  object 
 12  market_segment                  

Observamos que hay muchas variables categóricas, queremos codificarlas. Para ello, agrupamos todas las variables categóricas en una lista.

# PIPELINE

El pipeline crea y codifica las variables categóricas automáticamente.

In [4]:
#Definimos el X e Y
x = df.drop('adr', axis=1)
y = df['adr']

1. **Identifica las columnas automáticamente**

In [5]:
categoricas=x.select_dtypes(include=object).columns

In [6]:
numerical_cols = x.select_dtypes(exclude='object').columns

In [7]:
df[categoricas].info(),df[numerical_cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84080 entries, 0 to 84079
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   hotel                 84080 non-null  object
 1   arrival_date_month    84080 non-null  object
 2   meal                  84080 non-null  object
 3   country               84080 non-null  object
 4   market_segment        84080 non-null  object
 5   distribution_channel  84080 non-null  object
 6   reserved_room_type    84080 non-null  object
 7   deposit_type          84080 non-null  object
 8   customer_type         84080 non-null  object
dtypes: object(9)
memory usage: 5.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84080 entries, 0 to 84079
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   lead_time                       84080 non-null  int64  
 1   arriv

(None, None)

2. **Transformador Numérico**

In [8]:
numeric_transformer =Pipeline(steps=[
    ('scaler',StandardScaler())
])

3. **Transformador Categórico**

In [9]:
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore')) # 'ignore' evita errores en la API si llega algo nuevo
])

3. **Preprocesador**

In [10]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_cols),
        ('cat', categorical_transformer, categoricas)
    ])

**Pipeline Completo (Preprocesamiento + Modelo)**

In [11]:
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', xgboost.XGBRegressor(objective='reg:squarederror', n_jobs=-1, random_state=42))
])

# ENTRENAMIENTO DEL MODELO

Identificamos las variables "**x**" e "**y**". Dividimos el entrenamiento en un 80% del dataset para entrenarlo y el 20% restante como conjunto de prueba

In [12]:
x = df.drop('adr', axis=1)
y = df['adr']

In [13]:
x_train, x_test, y_train, y_test =train_test_split (x, y, test_size=0.2, random_state=42)

In [14]:
x_train.shape, x_test.shape

((67264, 27), (16816, 27))

Usamos el método XGBoost para entrenar nuestro modelo, que ya está conectado al Pipeline.

## Buscando mejores parámetros

In [15]:
from scipy.stats import randint, uniform

In [16]:
parametros = {
    # Número de árboles (más es mejor, pero más lento)
    'regressor__n_estimators': randint(200, 600),
    
    # Velocidad de aprendizaje (menor = más precisión, necesita más árboles)
    'regressor__learning_rate': uniform(0.01, 0.2), 
    
    # Profundidad máxima (controla la complejidad)
    'regressor__max_depth': randint(3, 10),
    
    # Peso mínimo para seguir dividiendo (evita overfitting)
    'regressor__min_child_weight': randint(1, 6),
    
    # Aleatoriedad para robustez (subsampleo de filas y columnas)
    'regressor__subsample': uniform(0.7, 0.3),
    'regressor__colsample_bytree': uniform(0.7, 0.3)
}

- randint= número enteros
- uniform= número decimal

In [17]:
from sklearn.model_selection import RandomizedSearchCV

In [18]:
rscv = RandomizedSearchCV(
    estimator=model_pipeline,
    param_distributions=parametros,
    n_iter=20,     #Probará 20 modelos diferentes
    cv=3,
    verbose=1,
    n_jobs=-1,
    scoring='r2'   #Optimiza R2
)

In [19]:
rscv.fit(x_train, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


,estimator,"Pipeline(step...=None, ...))])"
,param_distributions,"{'regressor__colsample_bytree': <scipy.stats....0015A1D7A2030>, 'regressor__learning_rate': <scipy.stats....0015A1D7A1850>, 'regressor__max_depth': <scipy.stats....0015A1D7A04A0>, 'regressor__min_child_weight': <scipy.stats....0015A1D7A07D0>, ...}"
,n_iter,20
,scoring,'r2'
,n_jobs,-1
,refit,True
,cv,3
,verbose,1
,pre_dispatch,'2*n_jobs'
,random_state,None
,error_score,nan


## Resultados y Guardado
Buscando qué modelo es el que mejor resultados dá y lo guardamos en "best_model"

In [20]:
best_model= rscv.best_estimator_

In [21]:
print("\n--- MEJORES RESULTADOS ---")
print(f"Mejor R2 (CV): {rscv.best_score_:.4f}")
print("Mejores Parámetros:", rscv.best_params_)


--- MEJORES RESULTADOS ---
Mejor R2 (CV): 0.8838
Mejores Parámetros: {'regressor__colsample_bytree': np.float64(0.903619762277746), 'regressor__learning_rate': np.float64(0.10895480323696125), 'regressor__max_depth': 9, 'regressor__min_child_weight': 3, 'regressor__n_estimators': 448, 'regressor__subsample': np.float64(0.8253252653734294)}


## Predicciones

In [22]:
from sklearn.metrics import accuracy_score

In [23]:
y_preds = best_model.predict(x_test)

In [24]:
comp=pd.DataFrame({"real": y_test, "preds": y_preds})

In [25]:
comp.head(20)

,real,preds
28096,48.20,47.437607
24867,55.00,54.598831
1291,113.70,168.349045
4990,233.00,229.257935
13178,68.74,77.162865
2362,43.20,57.861946
80926,170.50,142.512726
3804,47.00,44.483898
79692,139.50,140.321777
8619,180.00,170.668243


## Guardamos en archivos .pkl

In [26]:
import joblib

In [27]:
ruta = "C:\Proyecto-API-Hoteles/ML/"

In [28]:
joblib.dump(best_model, ruta+'modelo_pipeline_completo.joblib')

['C:\\Proyecto-API-Hoteles/ML/modelo_pipeline_completo.joblib']

# ESTADÍSTICAS DEL ERROR

In [29]:
comp['error'] = comp['real'] - comp['preds']
comp['error_abs'] = abs(comp['error'])

In [30]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [31]:
print("\n" + "=" * 60)
print("ESTADÍSTICAS DEL ERROR")
print("=" * 60)
print(f"Error medio: {comp['error'].mean():.2f}")
print(f"Error absoluto medio (MAE): {comp['error_abs'].mean():.2f}")
print(f"RMSE: {np.sqrt(mean_squared_error(comp['real'], comp['preds'])):.2f}")
print(f"R²: {r2_score(comp['real'], comp['preds']):.4f}")


ESTADÍSTICAS DEL ERROR
Error medio: -0.13
Error absoluto medio (MAE): 10.43
RMSE: 16.50
R²: 0.8920


## Conclusiones
El modelo que hemos entrenado con XGBoost, es un modelo en el que se puede considerar como bastante bueno para la predicción del **ADR (Average Daily Rate)**. Las métricas que hemos obtenido son las siguientes
- **Error Medio** = -0.13 Indica que el modelo no tiende a sobreestimar ni a subestimar el ADR de manera sistemática.
- **MAE** =10.43 El modelo se equivoca 10.43$ por noche. En nuestra predicción se considera bueno.
- **RMSE** =16.50 El error ponderado se mantiene en niveles razonables. Se considera aceptable.
- **R²** =0.8920 El modelo es muy bueno ya que el R² explica el 89.20% de la variabilidad del ADR real.